We want to use GBDX to obtain a bird's eye view of the degree of completeness of [OpenStreetMap (OSM)](https://www.openstreetmap.org) building footprints in different cities around the world. The goal is to use this information to intelligently direct mappers where they are most needed.

The idea is to run our unsupervised Land Use Land Cover (LULC) classification algorithm over these regions in order to 
classify each pixel as water, vegetation, clouds, soil, shadows and unclassified. By exclusion, the last category corresponds to materials like stone, cement and metal, which are used in buildings and roads, and can therefore be used as an approximation of built-up areas.

Overlaying the results of the algorithm with existing OSM building footprints, we can get an idea of OSM coverage. 

Create a GBDX interface using gbdxtools. You need your credentials to do this; you can find them under your profile on [gbdx.geobigdata.io](https://gbdx.geobigdata.io/login).

In [ ]:
import os
os.environ['GBDX_USERNAME'] = 
os.environ['GBDX_PASSWORD'] = 
os.environ['GBDX_CLIENT_ID'] =  
os.environ['GBDX_CLIENT_SECRET'] = 

import gbdxtools
gbdx = gbdxtools.Interface()

We've selected imagery for each city.

In [9]:
# catalog ids

cat_ids = {'nyc':         '104001001DB7BA00',
           'houston':     '104001001838A000',
           'la':          '104001001EBB4400', 
           'montreal':    '1040010023BEFD00', 
           'athens':      '104001001B6E1400', 
           'madrid':      '1040010019852500', 
           'nairobi':     '103001005C7E5400', 
           'amman':       '103001003E6FFC00', 
           'santiago':    '1040010029467C00',
           'bangkok':     '1030010063748E00',
           'cairo':       '1030010063AFF100',
           'osaka':       '10300100643CAC00',
           'buenosaires': '103001006414E800',
           'shanghai':    '1030010049993B00',
           'tehran':      '103001005ED0D000',
           'asuncion':    '103001005A8A6400',
           'ulaanbaatar': '103001005F575800',
           'perth':       '104001001D365400' }

cities = cat_ids.keys()

For each catalog id, we run a GBDX workflow which orders the raw image from the DG factory and then produces an orthorectified, atmospherically compensated 8-band image using the [AOP_Strip_Processor](http://gbdxdocs.digitalglobe.com/docs/advanced-image-preprocessor). Once the workflows complete, the images are stored under platform-stories/osm-lulc/images. This is a time-consuming step which you can skip as we've already run it for you.

In [ ]:
output_location = 'platform-stories/osm-lulc/images'   # where to save the images
wf_ids = {}

from os.path import join

for city in cities:
    
    # create order task
    # it the images are not on GBDX, this task will order them from the DG factory
    order = gbdx.Task('Auto_Ordering')
    order.inputs.cat_id = cat_ids[city]
    order.impersonation_allowed = True

    # run orthorectification and acomp
    aop_ms = gbdx.Task('AOP_Strip_Processor')
    aop_ms.inputs.data = order.outputs.s3_location.value
    aop_ms.inputs.bands = 'MS'
    aop_ms.inputs.enable_acomp = True
    aop_ms.inputs.enable_pansharpen = False
    aop_ms.inputs.enable_dra = False

    # define preprocessing workflow
    wf = gbdx.Workflow([order, aop_ms])

    # set output location 
    wf.savedata(aop_ms.outputs.data, join(output_location, city))

    # execute
    wf_ids[city] = wf.execute()
    

In [ ]:
# check status of workflows    
for city in cities:
    wf = gbdx.Workflow([])
    wf.id = wf_ids[city]
    print city, wf.id, wf.status

For each city, we've also determined a rectangular AOI. Find the coordinates below in W, S, E, N.

In [17]:
bboxes = {'nyc': (-73.95632922649384, 40.67078428278468, -73.89315783977509, 40.74625719581601),
          'houston': (-95.13090491294861, 29.719047995031016, -94.99769568443298, 29.901953768885594),
          'la': (-118.32255542278288, 34.15873756789489, -118.17252337932585, 34.28251185970165),
          'montreal': (-73.68750751018524, 45.62199932750296, -73.52236926555634, 45.755584499516516),
          'athens': (23.63831341266632, 38.01784624197521, 23.791778683662415, 38.11515296145808),
          'madrid': (-3.7636488676071167, 40.451429341017324, -3.6263197660446167, 40.5083584870881),
          'nairobi': (36.73872381448745, -1.3279983315316601, 36.88343435525894, -1.242017618358388),
          'amman': (35.79846203327179, 31.888389626458963, 35.97424328327179, 32.043632232381455),
          'santiago': (-70.74042499065399, -33.45030473839097, -70.58936297893524, -33.3886947889702),
          'bangkok': (100.44030010700226, 13.575965977708762, 100.5965119600296, 13.671059968841831),
          'cairo': (31.244285702705383, 30.045368026249772, 31.349342465400696, 30.11874679593054),
          'osaka': (135.4063493013382, 34.598781325037756, 135.48188030719757, 34.712028818736535),
          'buenosaires': (-58.50621789693832, -34.67611297581835, -58.39000314474106, -34.596736825050044),
          'shanghai': (121.50280773639678, 31.13559214758519, 121.60855114459991, 31.23486673223854),
          'tehran': (51.29124462604522, 35.712275224230865, 51.481788754463196, 35.8234243437396),
          'perth': (115.77790081501009, -31.958215741514543, 115.92449963092803, -31.80399586706599),
          'ulaanbaatar': (106.72447979450226, 47.8784119735556, 106.93768322467804, 47.93939724053319),
          'asuncion': (-57.67184436321258, -25.389250469786013, -57.524215579032905, -25.26729487165973)  
         }

We are now ready to deploy on GBDX. The following GBDX workflow does a few things:
- Runs LULC algorithm to create built-up mask for each area of interest as a black-and-white tif image. This is done with the task [lulc](https://github.com/PlatformStories/lulc).
- Downloads the OSM building footprints within the area of interest using the OSM Overpass API. We've created the task [download-osm-buildings](https://github.com/PlatformStories/download-osm-buildings) for this purpose, which outputs a geojson file with the building footprints. 
- Uploads the mask and the geojson to mapbox for visualization. We've created the task [upload-to-mapbox](https://github.com/PlatformStories/download-osm-buildings) for this purpose.

In [18]:
from os.path import join

wwf_ids = {}

# execute a workflow for each city
for city in cities:    
    
    # lulc
    lulc = gbdx.Task('lulc')
    lulc.inputs.image = join('s3://gbd-customer-data/32cbab7a-4307-40c8-bb31-e2de32f940c2/platform-stories/osm-lulc/images', city)
    lulc.inputs.unclassified = 'True'                    # get mask of unclassified class
    lulc.inputs.bbox = ','.join(map(str, bboxes[city]))  # extent of mask
    
    # download osm building footprints in bounding box
    dob = gbdx.Task('download-osm-buildings')
    dob.inputs.bbox = ','.join(map(str, bbox))

    # upload results to mapbox
    utom_mask = gbdx.Task('upload-to-mapbox')
    utom_mask.inputs.input = lulc.outputs.image.value
    utom_mask.inputs.tileset_name = 'osm-lulc-ras-' + city 
    utom_mask.inputs.token = mapbox_token    
    utom_footprints = gbdx.Task('upload-to-mapbox')
    utom_footprints.inputs.input = dob.outputs.geojson.value
    utom_footprints.inputs.tileset_name = 'osm-lulc-vec-' + city 
    utom_footprints.inputs.token = mapbox_token 
    
    # execute the workflow and save data on s3
    wf = gbdx.Workflow([lulc, dob, utom_mask, utom_footprints])
    output_location = join('platform-stories/trial-runs/osm-lulc', city)
    wf.savedata(lulc.outputs.image, join(output_location, 'mask'))
    wf.savedata(dob.outputs.geojson, join(output_location, 'geojson'))
    wwf_ids[city] = wf.execute()

In [20]:
# check status of workflows
for city in cities:
    wf = gbdx.Workflow([])
    wf.id = wwf_ids[city]
    print city, wf.id, wf.status

houston 4721511035795212728 {u'state': u'complete', u'event': u'succeeded'}
santiago 4721511038336219597 {u'state': u'complete', u'event': u'succeeded'}
shanghai 4721511040534352246 {u'state': u'complete', u'event': u'succeeded'}
la 4721511042788392266 {u'state': u'complete', u'event': u'succeeded'}
cairo 4721511045009778168 {u'state': u'complete', u'event': u'succeeded'}
madrid 4721511047195780891 {u'state': u'complete', u'event': u'succeeded'}
osaka 4721511050206380624 {u'state': u'complete', u'event': u'succeeded'}
buenosaires 4721511052592271826 {u'state': u'complete', u'event': u'succeeded'}
athens 4721511054826476755 {u'state': u'complete', u'event': u'succeeded'}
bangkok 4721511057099759538 {u'state': u'complete', u'event': u'succeeded'}
perth 4721511059355679532 {u'state': u'complete', u'event': u'succeeded'}
amman 4721511061509714340 {u'state': u'complete', u'event': u'succeeded'}
tehran 4721511063737872676 {u'state': u'complete', u'event': u'succeeded'}
nyc 472151106599725691

If you have a Mapbox account, you can check if your raster and vector tile sets have been successfully uploaded to mapbox at [https://www.mapbox.com/studio/tilesets/](https://www.mapbox.com/studio/tilesets/). We've created a demo html page with the results [here](http://gbdxstories.digitalglobe.com/pages/osm-lulc/cities.html). For each city, the built-up mask is shown in black-and-white and the buildings that we retrieved from the OSM Overpass API are shown in green.